In [1]:
import pandas as pd
import numpy as np
import math
from pandas_datareader.data import DataReader
import pandas_datareader
from datetime import datetime as dt
import os
import matplotlib.pyplot as plt
from datetime import timedelta
import time
from pandas_datareader.moex import MoexReader
import datetime

pd.set_option('display.max_columns', None)

/Users/denisbondarenko/miniconda3/lib/python3.7/site-packages/pandas_datareader/compat/__init__.py:7: FutureWarning: pandas.util.testing is deprecated. Use the functions in the public API at pandas.testing instead.
  from pandas.util.testing import assert_frame_equal


In [2]:
start_date = '2020-01-01'
end_date = '2022-01-01'

In [3]:
trades_info = pd.DataFrame(columns=['isin', 'mean_volume', 'sum_volume', 'mean_ntrades', 'sum_ntrades', 
                                    'days_trades', 'days_on_the_market'], dtype='object')

start_date = '2020-01-01'
end_date = '2021-12-31'
ratings = ['AAA', 'AA', 'A', 'BBB', 'BB', 'B']
for rating in ratings:
    bonds = pd.read_excel(rating+'_bonds.xlsx')
    print(rating)
    bonds['isin'] = bonds['ISIN']
    bonds.set_index('isin', inplace = True)
    bonds['emission_date'] = bonds[['Начало размещения', 'Окончание размещения']].max(axis = 1).astype('str')
    for isin in bonds['ISIN'].values:
        rd = MoexReader(isin, start_date, end_date)
        try:
            df = rd.read()

            date_em = bonds.loc[bonds.index[bonds['ISIN'] == isin]][['Начало размещения', 'Окончание размещения']].max(axis = 1)
            trades_info = trades_info.append({
                'rating':rating,
                'isin':isin,
                           'mean_volume':df['VOLUME'].mean(),
                           'sum_volume':df['VOLUME'].sum(),
                                             'mean_ntrades':df['NUMTRADES'].mean(),
                                             'sum_ntrades':df['NUMTRADES'].sum(),
            'days_trades':df[df['NUMTRADES']>0].shape[0],
            'days_on_the_market': np.busday_count(np.datetime64(bonds['emission_date'][isin]), 
                                                  np.datetime64(end_date))}, ignore_index=True)
        except:
            pass

trades_info['days_on_the_market'] = trades_info['days_on_the_market'].apply(lambda x: np.busday_count(np.datetime64(start_date), 
                                                  np.datetime64(end_date)) if x > np.busday_count(np.datetime64(start_date), 
                                                  np.datetime64(end_date)) else x)
trades_info

AAA
AA
A
BBB
BB
B


,isin,mean_volume,sum_volume,mean_ntrades,sum_ntrades,days_trades,days_on_the_market,rating
0,RU000A101PG7,979.173594,400482,4.777506,1954,336,420,AAA
1,RU000A101PF9,2009.009780,821685,1.449878,593,197,420,AAA
2,RU000A101PD4,2173.256724,888862,1.200489,491,195,420,AAA
3,RU000A103AT8,6799.421053,904323,35.526316,4725,133,135,AAA
4,RU000A0JS3U0,12.059524,6078,1.523810,768,263,522,AAA
...,...,...,...,...,...,...,...,...
634,RU000A1032S5,1612.352941,274100,148.688235,25277,170,171,B
635,RU000A1025E9,94.922601,30660,12.572755,4061,315,331,B
636,RU000A1005N2,306.250988,154963,10.650198,5389,455,522,B
637,RU000A100TD8,725.104743,366903,59.033597,29871,504,522,B


In [4]:
trades_info['%_of_days_traded'] = trades_info['days_trades'] / trades_info['days_on_the_market']
trades_info.head()

,isin,mean_volume,sum_volume,mean_ntrades,sum_ntrades,days_trades,days_on_the_market,rating,%_of_days_traded
0,RU000A101PG7,979.173594,400482,4.777506,1954,336,420,AAA,0.8
1,RU000A101PF9,2009.009780,821685,1.449878,593,197,420,AAA,0.469048
2,RU000A101PD4,2173.256724,888862,1.200489,491,195,420,AAA,0.464286
3,RU000A103AT8,6799.421053,904323,35.526316,4725,133,135,AAA,0.985185
4,RU000A0JS3U0,12.059524,6078,1.523810,768,263,522,AAA,0.503831


In [ ]:
trades_info.to_excel('trades_info.xlsx')